# Plugins

### C# Native Functions

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.45.0"
#r "nuget: Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.45.0"
#r "nuget: Microsoft.SemanticKernel.Yaml, 1.45.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.OpenApi, 1.45.0"

Installed Packages Microsoft.SemanticKernel, 1.45.0 Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.45.0 Microsoft.SemanticKernel.Yaml, 1.45.0

##### C# functions can be used to expose functionality to your model. This works well, when you already have an application built out, and would like to add AI to the app.

In [31]:
using System.ComponentModel;

public class MathFunctions
{
    [KernelFunction, Description("Calculates the square of a number.")]
    public static double square ([Description("The number to calculate square of")] double x)
    {
        return x * x;
    }

    [KernelFunction, Description("Calculates the cube of a number.")]
    public static double cube ([Description("The number to calculate cube of")] double x)
    {
        return x * x * x;
    }
}

 public class TimeInformation
    {
        [KernelFunction]
        [Description("Retrieves the current time in UTC.")]
        public string GetCurrentUtcTime() => DateTime.UtcNow.ToString("R");
    }

In [41]:
using Microsoft.SemanticKernel;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;


#!import config/Settings.cs 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
ILoggerFactory myLoggerFactory = NullLoggerFactory.Instance;
var builder = Kernel.CreateBuilder();
builder.Services.AddSingleton(myLoggerFactory); //Add logging!
builder.AddOpenAIChatCompletion(model, apiKey, orgId);
var kernel = builder.Build();
#pragma warning disable SKEXP0001 
// Lets the model choose the function to call based on the prompt
OpenAIPromptExecutionSettings settings = new() { FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() };

// We import the MathFunctions class to the kernel
// This will allow us to use the functions defined in the class
kernel.Plugins.AddFromType<MathFunctions>();
kernel.Plugins.AddFromType<TimeInformation>();

Console.WriteLine ( await kernel.InvokePromptAsync("What is the square of 25", new (settings)) );
Console.WriteLine ( await kernel.InvokePromptAsync("How many days until Christmas. Explain your thinking", new (settings)) );


The square of 25 is 625.
Today is April 24, 2025. Christmas is on December 25th, 2025.

To find out how many days are left until Christmas, we will count the days from April 24 to December 25 within the same year.

Here is the breakdown:
- April 24 to April 30: 6 days
- Full months from May to November: 7 months (31 days for May, 30 for June, 31 for July, 31 for August, 30 for September, 31 for October, and 30 for November). That's 214 days.
- December 1 to December 25: 25 days

Calculating the total: 6 (days in April) + 214 (days from May to November) + 25 (days in December) = 245 days.

Thus, there are 245 days remaining until Christmas.


#### What if you have a service you need to use to supply functionality? You can use OpenAPI!

In [7]:
using Microsoft.SemanticKernel;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;


#!import config/Settings.cs 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
ILoggerFactory myLoggerFactory = NullLoggerFactory.Instance;
var builder = Kernel.CreateBuilder();
builder.AddOpenAIChatCompletion(model, apiKey, orgId);
var kernel = builder.Build();

var stream = File.OpenRead("resources/api/repairs.json");
var plugin = await kernel.ImportPluginFromOpenApiAsync("RepairService", stream!);

kernel.Plugins.Add(plugin);

Error: (8,27): error CS1061: 'Kernel' does not contain a definition for 'ImportPluginFromOpenApiAsync' and no accessible extension method 'ImportPluginFromOpenApiAsync' accepting a first argument of type 'Kernel' could be found (are you missing a using directive or an assembly reference?)